<a href="https://colab.research.google.com/github/chqzeng/WaterSatOnCloud/blob/main/Tool4%20-%20Rayleigh%20Atmospheric%20Correction/Tool4_Rayleigh_correction_1_generating_LUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Py6S for Rayleigh Correction of level-1 LST8 data

Here we generate a lookup table to do a simple Rayleigh atmospheric correction for pixel values extracted in Tool 2.

Atmospheric reflectance is calculated in the absence of aerosol, and is removed from TOA reflectance according to the solar/observation geometry.

## Step 1 - Generating a Lookup Table

In [1]:
# Install Conda
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [ ]:
# Install Py6S from Conda
!conda install Py6S

In [2]:
# Create LUT
import pandas as pd
from Py6S import *

bands = {'B1': PredefinedWavelengths.LANDSAT_OLI_B1,
         'B2': PredefinedWavelengths.LANDSAT_OLI_B2,
         'B3': PredefinedWavelengths.LANDSAT_OLI_B3,
         'B4': PredefinedWavelengths.LANDSAT_OLI_B4,
         'B5': PredefinedWavelengths.LANDSAT_OLI_B5,
         'B6': PredefinedWavelengths.LANDSAT_OLI_B6,
         'B7': PredefinedWavelengths.LANDSAT_OLI_B7,
         'B8': PredefinedWavelengths.LANDSAT_OLI_B8,
         'B9': PredefinedWavelengths.LANDSAT_OLI_B9
         }

# solar and viewing zenith angles, 0 to 80 at an interval of 10 degrees
my_SZAs = list(range(0, 90, 10))
my_VZAs = list(range(0, 90, 10))

# relative azimuth, 0 to 180 at an interval of 10 degrees
my_RAAs = list(range(0, 190, 10))

# Output
LUT = []

# Compute atmospheric reflectance at different solar/viewing geometries
for k, v in bands.items():
    # k: B1, B2, etc.
    # v: value for wavelength
    print('\n========Band: ' + str(k))

    for my_SZA in my_SZAs:
        print('\nmy_SZA: ' + str(my_SZA) + '\n')
        for my_VZA in my_VZAs:
            print('my_VZA: ' + str(my_VZA))
            for my_RAA in my_RAAs:

                ### Running Py6S

                s = SixS()
                s.geometry = Geometry.User()

                s.geometry.solar_z = my_SZA
                s.geometry.solar_a = 0

                s.geometry.view_z = my_VZA
                s.geometry.view_a = my_RAA

                s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
                s.aero_profile = AeroProfile.PredefinedType(AeroProfile.NoAerosols)

                s.altitudes.set_target_custom_altitude(0)
                s.altitudes.set_sensor_satellite_level()
                s.ground_reflectance = GroundReflectance.HomogeneousLambertian(0)

                s.wavelength = Wavelength(v)
                s.run()

                # Calculate atmospheric reflectance
                R_atm = s.outputs.values['atmospheric_intrinsic_reflectance']
                LUT.append({'Band': k,
                                'SZA': my_SZA,
                                'VZA': my_VZA,
                                'RAA': my_RAA,
                                'R_atm': R_atm})

results = pd.DataFrame(LUT)


========Band: B1

my_SZA: 0

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 10

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 20

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 30

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 40

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 50

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 60

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 70

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA: 70
my_VZA: 80

my_SZA: 80

my_VZA: 0
my_VZA: 10
my_VZA: 20
my_VZA: 30
my_VZA: 40
my_VZA: 50
my_VZA: 60
my_VZA

# View and Save Results

In [3]:
print(results)

# Save the resulting LUT as a CSV
# results.to_csv('LUT.csv', index=False)

      Band  SZA  VZA  RAA  R_atm
0       B1    0    0    0  0.091
1       B1    0    0   10  0.091
2       B1    0    0   20  0.091
3       B1    0    0   30  0.091
4       B1    0    0   40  0.091
...    ...  ...  ...  ...    ...
13846   B9   80   80  140  0.023
13847   B9   80   80  150  0.025
13848   B9   80   80  160  0.026
13849   B9   80   80  170  0.028
13850   B9   80   80  180  0.028

[13851 rows x 5 columns]
